In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Configure gmaps to use your Google API key. You only have to do this once
gmaps.configure(api_key=g_key)

In [4]:
# Store the CSV saved/created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Bluff,NZ,-46.60,168.33,49.64,76,99,19.82,light rain,0.74,0
1,1,Atuona,PF,-9.80,-139.03,79.41,78,40,16.96,light rain,0.23,0
2,2,Pathein,MM,16.78,94.73,80.08,86,66,3.87,light rain,0.17,0
3,3,Smithers,CA,54.78,-127.17,59.00,58,90,11.41,overcast clouds,0.00,0
4,4,Pochutla,MX,15.74,-96.47,87.01,62,40,11.41,scattered clouds,0.00,0


In [5]:
city_data_df["Snow (inches)"].dtypes


dtype('int64')

In [6]:
# Get the maximum temperature. Replace negative temps with 0
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [8]:
# Ask the customer rain and snow preferences
rain_ask = input("Do you want it to be raining? (yes/no) ")
snow_ask = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [9]:

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df
if rain_ask == 'no' and snow_ask == 'no':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] == 0) & \
                                                  (preferred_cities_df["Snow (inches)"] == 0)]
elif rain_ask == 'yes' and snow_ask == 'no':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] > 0) & \
                                                  (preferred_cities_df["Snow (inches)"] == 0)]
elif rain_ask == 'no' and snow_ask == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] == 0) & \
                                                  (preferred_cities_df["Snow (inches)"] > 0)]
elif rain_ask == 'yes' and snow_ask == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] > 0) & \
                                                  (preferred_cities_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
4,4,Pochutla,MX,15.74,-96.47,87.01,62,40,11.41,scattered clouds,0.0,0
5,5,Sakakah,SA,29.97,40.21,91.40,18,0,10.29,clear sky,0.0,0
7,7,Kahului,US,20.89,-156.47,89.60,40,20,24.16,few clouds,0.0,0
8,8,Arraial Do Cabo,BR,-22.97,-42.03,75.20,69,0,13.87,clear sky,0.0,0
11,11,Vaini,TO,-21.20,-175.20,73.40,78,75,14.99,broken clouds,0.0,0
14,14,Harnai,PK,30.10,67.94,89.78,25,0,3.31,clear sky,0.0,0
16,16,Biltine,TD,14.53,20.92,83.44,51,33,2.89,scattered clouds,0.0,0
24,24,Mandali,IQ,33.75,45.55,94.50,12,0,14.09,clear sky,0.0,0
28,28,Keti Bandar,PK,24.14,67.45,84.67,77,100,18.79,overcast clouds,0.0,0
32,32,Wilson,US,36.15,-86.28,91.00,43,20,13.87,few clouds,0.0,0


In [10]:
preferred_cities_df.count()

City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
Rain (inches)          243
Snow (inches)          243
dtype: int64

In [11]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64

In [12]:
# Create DataFrame called cities_hotel_df that adds a hotel in the city 
cities_hotel_df = preferred_cities_df.copy()
cities_hotel_df["Hotel Name"] = ""
cities_hotel_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
4,4,Pochutla,MX,15.74,-96.47,87.01,62,40,11.41,scattered clouds,0.0,0,
5,5,Sakakah,SA,29.97,40.21,91.40,18,0,10.29,clear sky,0.0,0,
7,7,Kahului,US,20.89,-156.47,89.60,40,20,24.16,few clouds,0.0,0,
8,8,Arraial Do Cabo,BR,-22.97,-42.03,75.20,69,0,13.87,clear sky,0.0,0,
11,11,Vaini,TO,-21.20,-175.20,73.40,78,75,14.99,broken clouds,0.0,0,
14,14,Harnai,PK,30.10,67.94,89.78,25,0,3.31,clear sky,0.0,0,
16,16,Biltine,TD,14.53,20.92,83.44,51,33,2.89,scattered clouds,0.0,0,
24,24,Mandali,IQ,33.75,45.55,94.50,12,0,14.09,clear sky,0.0,0,
28,28,Keti Bandar,PK,24.14,67.45,84.67,77,100,18.79,overcast clouds,0.0,0,
32,32,Wilson,US,36.15,-86.28,91.00,43,20,13.87,few clouds,0.0,0,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [14]:
# Iterate through the DataFrame.
for index, row in cities_hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        cities_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
cities_hotel_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
4,4,Pochutla,MX,15.74,-96.47,87.01,62,40,11.41,scattered clouds,0.0,0,Hotel Posada San Jose
5,5,Sakakah,SA,29.97,40.21,91.40,18,0,10.29,clear sky,0.0,0,Fakhamat Al Orjoana Apartment
7,7,Kahului,US,20.89,-156.47,89.60,40,20,24.16,few clouds,0.0,0,Maui Seaside Hotel
8,8,Arraial Do Cabo,BR,-22.97,-42.03,75.20,69,0,13.87,clear sky,0.0,0,Pousada Porto Praia
11,11,Vaini,TO,-21.20,-175.20,73.40,78,75,14.99,broken clouds,0.0,0,Keleti Beach Resort
14,14,Harnai,PK,30.10,67.94,89.78,25,0,3.31,clear sky,0.0,0,Shah g hotel
16,16,Biltine,TD,14.53,20.92,83.44,51,33,2.89,scattered clouds,0.0,0,Hôtel Trois Étoiles de Biltine
24,24,Mandali,IQ,33.75,45.55,94.50,12,0,14.09,clear sky,0.0,0,بيت خالد امين
28,28,Keti Bandar,PK,24.14,67.45,84.67,77,100,18.79,overcast clouds,0.0,0,Haji laloo memon house
32,32,Wilson,US,36.15,-86.28,91.00,43,20,13.87,few clouds,0.0,0,Holiday Inn Express & Suites Lebanon


In [16]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
cities_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# add a pop-up marker for each city that displays the hotel name, city name, country,
# current weather description and maximum temperature.
# Define template for pop-up box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row. use the iterrows() function to get the index and data in the row to add to the marker.
# hotel_info is a list
hotel_info = [info_box_template.format(**row) for index, row in cities_hotel_df.iterrows()]


In [18]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# Get the latitude and longitude.
locations = cities_hotel_df[["Lat", "Lng"]]
max_temp = cities_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))